In [1]:
!pip install PyYAML==5.4.1
import yaml

In [ ]:
! pip install datasets transformers rouge-score nltk

In [1]:
from torch.nn import CrossEntropyLoss, MSELoss

In [2]:
import transformers

print(transformers.__version__)

4.11.1


# Fine-tuning a model on a summarization task

In [3]:
#model_checkpoint = "facebook/bart-base"
model_checkpoint = "facebook/bart-large"

## Loading the dataset

In [4]:
import datasets
from datasets import load_dataset, load_metric

#train_dataset = load_dataset('json', data_files='kp20k_BART_train_finetune.src')
#valid_dataset = load_dataset('json', data_files='kp20k_BART_valid_finetune.src')
#test_dataset = load_dataset('json', data_files='kp20k_BART_test_finetune.src')

#raw_datasets = datasets.DatasetDict({"train":train_dataset,"valid":valid_dataset,"test":test_dataset})
raw_datasets = load_dataset('json',data_files={'train': 'kp20k_KPBL_train(1028).json', 'valid': 'kp20k_KPBL_valid(1028).json', 'test': 'kp20k_KPBL_test(1028).json'})
#datasets.DatasetDict.from_json(path_or_paths = 'kp20k_smalltrain_finetune.json') # , features = ['document', 'summary', 'id'])
#raw_datasets = load_dataset(datasets.DatasetDict.from_json(path_or_paths = 'kp20k_smalltrain_finetune.json')) # , features = ['document', 'summary', 'id']))
#from_json
metric = load_metric("rouge")

Using custom data configuration default-937cce82eeeafca8
Reusing dataset json (C:\Users\delab-win\.cache\huggingface\datasets\json\default-937cce82eeeafca8\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 513918
    })
    valid: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 19992
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 19987
    })
})

In [6]:
raw_datasets["train"][0]

{'document': 'joint source channel authentication resource allocation and unequal authenticity protection for multimedia over wireless networks . there have been increasing concerns about the security issues of wireless transmission of multimedia in recent years . wireless networks , by their nature , are more vulnerable to external intrusions than wired ones . many applications demand authenticating the integrity of multimedia content delivered wirelessly . in this work , we describe a framework for jointly coding and authenticating multimedia to be delivered over heterogeneous wireless networks . we firstly introduce a novel concept called unequal authenticity protection ( uap ) , which unequally allocate resources to achieve an optimal authentication result . we then consider integrating uap with specific source and channel coding models , to obtain optimal end to end quality by the means of joint source channel authentication analysis . lastly , we present an implementation of the 

In [7]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"])

,document,id,summary
0,"automatic code mapping on an intelligent memory architecture . this paper presents an algorithm to automatically map code on a generic intelligent memory system that consists of a high end host processor and a simpler memory processor . to achieve high performance with this type of architecture , the code needs to be partitioned and scheduled such that each section is assigned to the processor on which it runs most efficiently , in addition , the two processors should overlap their execution as much as possible . with our algorithm , applications are mapped fully automatically using both static and dynamic information . using a set of standard applications and a simulated architecture , we obtain average speedups of 1 . 7 for numerical applications and 1 . 2 for nonnumerical applications over a single host with plain memory , the speedups are very close and often higher than ideal speedups on a more expensive multiprocessor system composed of two identical host processors . our work shows that heterogeneity can be cost effectively exploited and represents one step toward effectively mapping code on heterogeneous intelligent memory systems",365710,"[""intelligent memory architecture"", ""processing in memory"", ""compilers"", ""adaptive execution"", ""performance prediction"", ""heterogeneous system""]"
1,"ftc floating precision texture compression . this paper presents new fixed rate texture compression systems for rgb and rgba images , which use variable precision differential encoding for the color codewords , resulting in reduced compression artifacts , in particular for smooth color variations . experiments using hundreds of textures show that they achieve higher quality than the commonly used s3tc at the same compression ratio or ( for rgba images ) similar quality to s3tc at twice the compression ratio . in hardware implementations a large part of decompression hardware can be shared with existing common systems , minimizing the amount of additional hardware needed .",18509,"[""image compression"", ""texture"", ""texture compression"", ""graphics hardware""]"
2,"definition of an educational administration automated system . this work presents the definition of an automated system of administration for a superior education institution . the objective of this system is to improve the administrative system and information service that it is offered for this superior education institute escuela de computacion universidad central de venezuela . the system allows to carry out academic requirements to professors , administrative personnel and students , by using a computer network . the steps of each requirement will be controlled automatically in the corresponding instances according to the defined flow for each particular process . the system is conceived as the integration of a group of interactive subsystems . to communicate to each other , each subsystem share a friendly interface with a web site . this web site is the entry to institution ' s intranet and extranet",337582,"[""intranet"", ""process"", ""reengineering"", ""redesign"", ""workflow""]"
3,"origins of illusory percepts in digital images . here we show the relation between illusory percepts and statistical regularities across scales and orientations . to this aim , the performance of a computational model for the partitioning of statistical regularities is analyzed on several tasks such as long range boundary completion , phase induced contour detection , as well as shape and size illusions . the system for the automatically learned partitioning of statistical regularities in 2d images , is based on a sophisticated , band pass , filtering operation , with fixed scale and orientation sensitivity . experimental results are provided to illustrate this analysis on several examples ( i ) kanizsa type subjective figures ( ii ) phase induced subjective contours ( iii ) the zllner illusion and ( iv ) the mllerlyer illusion",150607,"[""illusory per

In [10]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [11]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

In [8]:
from transformers import AutoTokenizer, BartTokenizerFast
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)#,add_prefix_space=True

In [9]:
if model_checkpoint in ["facebook/bart-large"]:
    prefix = "summarize: "
else:
    prefix = ""

In [10]:
max_input_length = 800
max_target_length = 200

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length)#, is_split_into_words= True,truncation=True

    model_inputs["labels"] =eec labels["input_ids"]
    return model_inputs

In [21]:
>>> tokenizer("scatternet")['input_ids']

[0, 3866, 415, 18995, 594, 2]

wandb: Network error resolved after 0:03:04.896992, resuming normal operation.
wandb: 500 encountered ({"error":"Error 1040: Too many connections"}), retrying request
wandb: Network error resolved after 0:01:18.315582, resuming normal operation.
wandb: Network error resolved after 0:03:21.311012, resuming normal operation.


In [11]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 18581, 3916, 2072, 35, 2660, 1300, 4238, 24790, 5799, 12278, 8, 37563, 21341, 2591, 13, 15778, 81, 6955, 4836, 479, 89, 33, 57, 2284, 1379, 59, 5, 573, 743, 9, 6955, 9235, 9, 15778, 11, 485, 107, 479, 6955, 4836, 2156, 30, 49, 2574, 2156, 32, 55, 4478, 7, 6731, 19567, 26841, 87, 26977, 1980, 479, 171, 2975, 1077, 36106, 13659, 5, 7066, 9, 15778, 1383, 2781, 8869, 12445, 479, 11, 42, 173, 2156, 52, 6190, 10, 7208, 13, 13521, 25776, 8, 36106, 13659, 15778, 7, 28, 2781, 81, 39872, 42019, 6955, 4836, 479, 52, 78, 352, 6581, 10, 5808, 4286, 373, 37563, 21341, 2591, 36, 1717, 1115, 4839, 2156, 61, 46321, 2368, 25915, 1915, 7, 3042, 41, 19329, 24790, 898, 479, 52, 172, 1701, 22688, 1717, 1115, 19, 2167, 1300, 8, 4238, 25776, 3092, 2156, 7, 6925, 19329, 253, 7, 253, 1318, 30, 5, 839, 9, 2660, 1300, 4238, 24790, 1966, 479, 94, 352, 2156, 52, 1455, 41, 5574, 9, 5, 1850, 2660, 25776, 8, 24790, 467, 15, 10, 8212, 1236, 41191, 740, 15362, 479, 14073, 775, 8085, 14, 5, 1850, 1548,

In [11]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\delab-win\.cache\huggingface\datasets\json\default-937cce82eeeafca8\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-ff0a97039e4834c2.arrow


  0%|          | 0/20 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Loading cached processed dataset at C:\Users\delab-win\.cache\huggingface\datasets\json\default-937cce82eeeafca8\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-669622cd76369933.arrow


## Fine-tuning the model

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [13]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "KPBL-1028",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=30000,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
class BARTTrainerCE(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\delab-
[nltk_data]     win\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We can now finetune our model by just calling the `train` method:

In [18]:
trainer.train(resume_from_checkpoint = True)# resume_from_checkpoint = True

Loading model from KPBL-1028\checkpoint-450000).
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: document, id, summary.
***** Running training *****
  Num examples = 513918
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 642400
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 450000
  Will skip the first 3 epochs then the first 64560 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/64560 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: adiya (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Didn't find an RNG file, if you are resuming a training that was launched in a distributed fashion, reproducibility is not guaranteed.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
4,0.997100,1.333627,48.531600,25.725300,39.995600,40.024200,18.666900
5,0.916800,1.461712,48.544600,25.636900,40.035800,40.056500,18.698300


Saving model checkpoint to KPBL-1028\checkpoint-480000
Configuration saved in KPBL-1028\checkpoint-480000\config.json
Model weights saved in KPBL-1028\checkpoint-480000\pytorch_model.bin
tokenizer config file saved in KPBL-1028\checkpoint-480000\tokenizer_config.json
Special tokens file saved in KPBL-1028\checkpoint-480000\special_tokens_map.json
Deleting older checkpoint [KPBL-1028\checkpoint-390000] due to args.save_total_limit
Saving model checkpoint to KPBL-1028\checkpoint-510000
Configuration saved in KPBL-1028\checkpoint-510000\config.json
Model weights saved in KPBL-1028\checkpoint-510000\pytorch_model.bin
tokenizer config file saved in KPBL-1028\checkpoint-510000\tokenizer_config.json
Special tokens file saved in KPBL-1028\checkpoint-510000\special_tokens_map.json
Deleting older checkpoint [KPBL-1028\checkpoint-420000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have 

TrainOutput(global_step=642400, training_loss=0.2835925570876334, metrics={'train_runtime': 86091.3986, 'train_samples_per_second': 29.847, 'train_steps_per_second': 7.462, 'total_flos': 1.8166733799227228e+18, 'train_loss': 0.2835925570876334, 'epoch': 5.0})

In [19]:
#trainer.push_to_hub()
trainer.save_model(output_dir='PS-BL-Finetuned-1028')

Saving model checkpoint to PS-BL-Finetuned-1028
Configuration saved in PS-BL-Finetuned-1028\config.json
Model weights saved in PS-BL-Finetuned-1028\pytorch_model.bin
tokenizer config file saved in PS-BL-Finetuned-1028\tokenizer_config.json
Special tokens file saved in PS-BL-Finetuned-1028\special_tokens_map.json
wandb: Network error resolved after 0:03:12.371202, resuming normal operation.
